# Support Vector Machines


Imports


In [133]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import pandas as pd
from binning import bin

Read the train and test datasets


In [2]:
train_df = pd.read_csv("Training_set.csv")
test_df = pd.read_csv("Testing_set.csv")

Get the training features and label


In [26]:
X_train = train_df.iloc[:, :-1]
y_train = train_df.iloc[:, -1]

Get the testing features and label


In [27]:
X_test = test_df.iloc[:, :-1]
y_test = test_df.iloc[:, -1]

Bin the label


In [156]:
y_train_binned = bin(y_train.to_numpy())
y_test_binned = bin(y_test.to_numpy())

Initialize an SVM classifier


In [146]:
svc = LinearSVC(dual=False, verbose=10)

Train the model


In [ ]:
svc.fit(X_train, y_train_binned)

Make the predictions on the test set


In [148]:
y_pred = svc.predict(X_test)

Evaluate the predictions


In [151]:
print(classification_report(y_true=y_test_binned, y_pred=y_pred, zero_division=0))

              precision    recall  f1-score   support

           0    1.00000   0.00009   0.00017     11718
           1    0.46732   0.84371   0.60148     42216
           2    0.41553   0.25817   0.31847     32447
           3    0.00000   0.00000   0.00000      9998

    accuracy                        0.45649     96379
   macro avg    0.47071   0.27549   0.23003     96379
weighted avg    0.46617   0.45649   0.37070     96379

